# 2014-2018 ACS Data by Neighborhood Tabulation Area (NTA)

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas.api.types import CategoricalDtype

from shapely.geometry import Point, Polygon, MultiPolygon
import geopandas as gpd

In [2]:
def colpercent(df):
    print("Total NaN in Dataframe: " , df.isnull().sum().sum())
    print("Percent Missingness in Dataframe: ", 100*df.isnull().sum().sum()/(len(df.index)*len(df.columns)))
    print('-'*55)
    percentnulldf = df.isnull().sum()/(df.isnull().sum()+df.notna().sum())
    print("Percent Missingness by Columns:")
    print(100*percentnulldf[percentnulldf>0].sort_values(ascending=False))
    
def data_eval(df):
    for i in range(len(df.columns)):
        print('-'*50)
        print('Column Name: ', df.columns[i])
        if (df[df.columns[i]].dtypes == 'float64' or df[df.columns[i]].dtypes == 'int64') and df[df.columns[i]][df[df.columns[i]]<0].count()>0:
            print('Number of negatives: ', df[df.columns[i]][df[df.columns[i]]<0].count())
        if df[df.columns[i]][df[df.columns[i]]=='None'].count() > 0:
            print('Number of None strings: ', df[df.columns[i]][df[df.columns[i]]=='None'].count())
        if df[df.columns[i]][df[df.columns[i]]==''].count() > 0:
            print('Number of empty strings: ', df[df.columns[i]][df[df.columns[i]]==''].count())
        else:
            print('Column ' + str(i) + ' has no negatives, empty strings or Nones')

## Import and load data

In [3]:
root_path = '../../data/00_raw/'
nta_path = root_path + 'econ_2018_acs5yr_nta.xlsx'
ntageo_path = root_path + 'nta_acs_201418.geojson'
cb_path = root_path + 'citibike_feature_engineered_final_v2.csv'

In [4]:
acsnta_raw = pd.read_excel(nta_path,
                          usecols = 'B:D, J, O, T, Y, AD, AI, CL, CQ, CV, DA, DF, DK, DP, DU, DZ, EE, EJ, EO, ET, EY, HV, IA, IF, IK, IP, IU, IZ, IE, JE, JJ, JO, JT, JY, KD, KI, KN, KS, KX, LC, PD, PI, PN, UI, UN, US, UX')

In [5]:
acsnta = acsnta_raw.copy()

In [6]:
acsnta = acsnta.drop(columns=['PrvWSWrkrZ'])

Rename columns:

In [7]:
column_rename = {'GeogName': 'nta_name',
                 'GeoID': 'nta_code',
                 'Borough': 'borough',
                 'LFE': 'in_labor_force',
                 'CvLF1E': 'civilian_labor_force',
                 'CvEm16pl1E': 'employed',
                 'CvLFUEm1E': 'unemployed',
                 'LFArmdFE': 'armed_forces',
                 'NLF1E': 'not_in_labor_force',
                 'Wrkr16plE': 'workers_16+',
                 'CW_DrvAlnE': 'vehicle',
                 'CW_CrpldE': 'carpool',
                 'CW_PbTrnsE': 'public_transit',
                 'CW_WlkdE': 'walking',
                 'CW_OthE': 'other_commuting_means',
                 'CW_WrkdHmE': 'work_at_home',
                 'MnTrvTmE': 'mean_commuting_time_(min)',
                 'CvEm16pl2E': 'occ_civilian_employed_16+',
                 'MgBSciArtE': 'mng_biz_sci_arts',
                 'SrvcE': 'service',
                 'SalesOffE': 'sales_office',
                 'NRCnstMntE': 'natres_construct_maint',
                 'PrdTrnsMME': 'prod_transport_moving',
                 'CvEm16pl4E': 'class_civilian_employed_16+',
                 'PrvWSWrkrE': 'salary_workers',
                 'GvtWrkrE': 'govt_workers',
                 'SlfEmNIncE': 'self-employed',
                 'UpdFmWrkrE': 'unpaid_family_workers',
                 'HH2E': 'total_households',
                 'HHIU10E': 'hincome_10K_under',
                 'HHI10t14E': 'hincome_10K_15K',
                 'HHI15t24E': 'hincome_15K_25K',
                 'HHI25t34E': 'hincome_25K_35K',
                 'HHI35t49E': 'hincome_35K_50K',
                 'HHI50t74E': 'hincome_50K_75K',
                 'HHI75t99E': 'hincome_75K_100K',
                 'HI100t149E': 'hincome_100K_150K',
                 'HI150t199E': 'hincome_150K_200K',
                 'HHI200plE': 'hincome_200K_more',
                 'MdHHIncE': 'median_hincome',
                 'MnHHIncE': 'mean_hincome',
                 'MdEWrkE': 'median_earnings',
                 'MdEMFTWrkE': 'median_earnings_male',
                 'MdEFFTWrkE': 'median_earnings_female',
                 'FamPvUE': 'all_families',
                 'FamBwPvE': 'all_families_below_poverty',
                 'PopPvU1E': 'pop_poverty_status_determ',
                 'PBwPvE': 'pop_below_poverty'}

In [8]:
acsnta = acsnta.rename(columns=column_rename)

Exclude Staten Island:

In [9]:
acsnta = acsnta[acsnta.borough != 'Staten Island']

Fill missing values:

In [10]:
acsnta = acsnta.fillna(0)

Save as csv:

In [33]:
acsnta.to_csv('../../data/00_raw/acsnta.csv', index=False)

In [34]:
acsnta

,nta_name,nta_code,borough,in_labor_force,civilian_labor_force,employed,unemployed,armed_forces,not_in_labor_force,workers_16+,...,hincome_200K_more,median_hincome,mean_hincome,median_earnings,median_earnings_male,median_earnings_female,all_families,all_families_below_poverty,pop_poverty_status_determ,pop_below_poverty
0,Allerton-Pelham Gardens,BX31,Bronx,15835,15835,14887,948,0,10779,14471,...,707,71414.0,88250.0,39182.0,50078.0,49112.0,7344,601,30243,3434
1,Bedford Park-Fordham North,BX05,Bronx,29295,29269,25755,3514,26,14923,24837,...,353,37282.0,54033.0,25194.0,35131.0,33773.0,13366,3979,57380,18625
2,Belmont,BX06,Bronx,12206,12191,10284,1907,15,10208,9854,...,42,28484.0,38077.0,16113.0,32700.0,28710.0,5856,1967,26275,10141
3,Bronxdale,BX07,Bronx,18614,18614,16251,2363,0,12042,15816,...,184,38587.0,50720.0,29471.0,39626.0,39773.0,9260,2372,38787,10503
4,Claremont-Bathgate,BX01,Bronx,13226,13226,11111,2115,0,11830,10675,...,34,25061.0,37407.0,20295.0,37323.0,33137.0,7621,3307,34558,15763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,St. Albans,QN08,Queens,28577,28518,26263,2255,59,14409,25461,...,1106,77454.0,93423.0,39469.0,50791.0,46456.0,11913,820,52509,4316
172,Steinway,QN72,Queens,29530,29483,28143,1340,47,12467,26865,...,1480,74387.0,96276.0,45980.0,58015.0,55890.0,10514,685,48174,4742
173,Whitestone,QN49,Queens,15715,15676,14834,842,39,10450,14458,...,1174,86871.0,106229.0,43608.0,70534.0,55484.0,8559,438,31782,1831
174,Woodhaven,QN53,Queens,31244,31230,29009,2221,14,17285,28126,...,866,71580.0,85858.0,32483.0,43878.0,41348.0,13539,1374,60378,7381


In [11]:
ntageo_path = '../../data/00_raw/nta_acs_201418.geojson'
nta_geo = gpd.read_file(ntageo_path)

# Citibike dataset

Import and load data:

In [12]:
cb = pd.read_csv(cb_path, parse_dates=['starttime', 'stoptime'])

Check for missing values:

In [13]:
cb.isnull().sum()[cb_raw.isnull().sum() > 0]

startneighborhood    275380
endneighborhood      277796
startboro            275380
dtype: int64

Drop duplicates:

In [14]:
start_stations = cb[['startstationid','startstationlatitude', 'startstationlongitude']].drop_duplicates('startstationid')
end_stations = cb_raw[['endstationid','endstationlatitude', 'endstationlongitude']].drop_duplicates('endstationid')
start_stations.columns = ['station_id', 'latitude', 'longitude']
end_stations.columns = ['station_id', 'latitude', 'longitude']
stations = pd.concat([start_stations, end_stations]).drop_duplicates('station_id')

Checking if stations are inside Neighborhood Tabulation Area:

In [15]:
stations['point'] = [Point(stations.iloc[i, 2], stations.iloc[i, 1]) for i in range(len(stations))]

In [16]:
stations = stations.drop(columns=['latitude', 'longitude'])

In [17]:
nta_code_geometry = nta_geo[['NTACode', 'geometry']]

In [18]:
temp = pd.DataFrame(columns = ['station_id', 'nta_code'])

for station in range(len(stations)):
    for nta_block in range(len(nta_code_geometry)):
        if stations.iloc[station,1].within(nta_code_geometry.iloc[nta_block, 1]):
            temp = temp.append({'station_id':stations.iloc[station,0], 'nta_code':nta_code_geometry.iloc[nta_block, 0]}, ignore_index=True)

Merge CitiBike data with NTA data:

In [19]:
acsnta2 = acsnta.merge(temp, how='right', on='nta_code')

In [20]:
acsnta2.columns

Index(['nta_name', 'nta_code', 'borough', 'in_labor_force',
       'civilian_labor_force', 'employed', 'unemployed', 'armed_forces',
       'not_in_labor_force', 'workers_16+', 'vehicle', 'carpool',
       'public_transit', 'walking', 'other_commuting_means', 'work_at_home',
       'mean_commuting_time_(min)', 'occ_civilian_employed_16+',
       'mng_biz_sci_arts', 'service', 'sales_office', 'natres_construct_maint',
       'prod_transport_moving', 'class_civilian_employed_16+',
       'salary_workers', 'govt_workers', 'self-employed',
       'unpaid_family_workers', 'total_households', 'hincome_10K_under',
       'hincome_10K_15K', 'hincome_15K_25K', 'hincome_25K_35K',
       'hincome_35K_50K', 'hincome_50K_75K', 'hincome_75K_100K',
       'hincome_100K_150K', 'hincome_150K_200K', 'hincome_200K_more',
       'median_hincome', 'mean_hincome', 'median_earnings',
       'median_earnings_male', 'median_earnings_female', 'all_families',
       'all_families_below_poverty', 'pop_poverty_sta

Save as csv:

In [21]:
acsnta2.to_csv('acsnta_with_station_id.csv', index=False)

Sampling the downsampled file for more efficient processing analysis:

In [22]:
cb = cb_raw.copy()

In [24]:
# citidb2 = citidb[citidb['starttime'].dt.year <= 2018]

In [25]:
# citidb2 = citidb2[citidb2['starttime'].dt.year >= 2014]

In [23]:
acsnta2_start = acsnta2.copy()
acsnta2_end = acsnta2.copy()

In [24]:
acsnta2_start.columns = ['start_' + column for column in acsnta2.columns if column != 'station_id'] + ['station_id']
acsnta2_end.columns = ['end_' + column for column in acsnta2.columns if column != 'station_id'] + ['station_id']

Merge files:

In [25]:
citidb2 = citidb2.merge(acsnta2_start, left_on='startstationid', right_on='station_id')

In [26]:
citidb2 = citidb2.merge(acsnta2_end, left_on='endstationid', right_on='station_id')

Cleaning:

In [27]:
citidb2 = citidb2.drop(columns = ['station_id_x', 'station_id_y'])

In [28]:
sample = citidb2.sample(n=int(len(citidb)*0.08))

Save as csv:

In [29]:
sample.to_csv('sample_citibike_with_acsnta.csv', index=False)

In [30]:
citidb2.to_csv('citibike_with_acsnta.csv', index=False)